In [17]:
from __future__ import annotations
from abc import ABCMeta, abstractmethod
from dataclasses import dataclass, field

import numpy as np
import typing as tp


Tn = tp.TypeVar("Tn")

@dataclass
class DataNode(tp.Generic[Tn]):
    data: tp.Any = None
    name: str | None = None
    children: list[Tn] | None = None
    
    
class FunctionNode(tp.Generic[Tn]):
    def __init__(
        self, 
        name: str | None = None, 
        children: list[Tn] | None = None
    ) -> None:
        self.name = name
        if children is None:
            self.children = []
        else:
            self.children = children
            
# Tn = tp.TypeVar("Tn", bound="BaseNode")
# 
# class BaseNode(tp.Generic[Tn]):
#     pass
# 
# @dataclass
# class DataNode(BaseNode[Tn]):
#     data: tp.Any = None
#     name: str | None = None
#     children: list[Tn] | None = None
#     
#     
# class FunctionNode(BaseNode[Tn]):
#     def __init__(
#         self, 
#         name: str | None = None, 
#         children: list[Tn] | None = None
#     ) -> None:
#         self.name = name
#         if children is None:
#             self.children = []
#         else:
#             self.children = children
# 


In [44]:
import uuid
import numpy as np
import pandas as pd


@dataclass
class ID(DataNode["ID"]):
    data: str = str(uuid.uuid4())

@dataclass
class Array(DataNode):
    data: np.ndarray
    children: None = None
    
    def min(self) -> float:
        return float(self.data.min())
    
    def max(self) -> float:
        return float(self.data.max())
    
    def mean(self) -> float:
        return float(self.data.mean())
    
    def std(self) -> float:
        return float(self.data.std())
    
    
@dataclass
class Energy(Array):
    name: str = "energy"

@dataclass
class Objective(Array):
    name: str = "objective"

@dataclass
class ConstraintViolation(Array):
    def __post_init__(self):
        if self.name is None:
            raise NameError("Attribute 'name' is None. Please set a name.")


class Record(FunctionNode[DataNode]):
    def __init__(self, name: str | None = None, children: list[DataNode] | None = None) -> None:
        super().__init__(name, children)
    
    def to_dict(self) -> dict:
        if self.children is None:
            raise TypeError("Value of attribute 'children' is None. Therefore, it cannot be converted to dict.")
        else:
            return {node.name: node.data for node in self.children}
            
    def to_series(self) -> pd.Series:
        return pd.Series(self.to_dict())
    

class Table(FunctionNode[Record]):
    def __init__(self, name: str | None = None, children: list[Record] | None = None) -> None:
        super().__init__(name, children)
    
    
    def to_dataframe(self) -> pd.DataFrame:
        return pd.concat([record.to_series() for record in self.children], axis=1).T
    
class Artifact(FunctionNode[Record]):
    def __init__(self, name: str | None = None, children: list[Record] | None = None) -> None:
        super().__init__(name, children)
    
    def to_dict(self):
        pass
    


In [19]:
i = ID()
print(i.data)
print(i.children)
print(i.name)

6e313955-bb9e-4b9d-b66b-860a2c14f291
None
None


In [20]:
array = Array()
print(array.data)
print(array.children)
print(array.name)

None
None
None


In [6]:
import jijzept as jz
import jijmodeling as jm

sampler = jz.JijSASampler(config="/home/d/.jijzept/config.toml")

problem = jm.Problem("sample")
x = jm.Binary("x", 5)
problem += x[:]
problem += jm.Constraint("onehot", x[:] == 1)

res = sampler.sample_model(problem, {})

In [21]:
res.feasible()

SampleSet(record=Record(solution={'x': [(([4],), [1], (5,))]}, num_occurrences=[1]), evaluation=Evaluation(energy=[0.0], objective=[1.0], constraint_violations={'onehot': [0.0]}, penalty={}), measuring_time=MeasuringTime(solve=SolvingTime(preprocess=None, solve=0.009687423706054688, postprocess=None), system=SystemTime(post_problem_and_instance_data=0.8192272186279297, request_queue=0.351177453994751, fetch_problem_and_instance_data=None, fetch_result=2.4272255897521973, deserialize_solution=0.0001010894775390625), total=3.6054487228393555))

In [22]:
b_id = ID(name="benchmark_id")
energy = Energy(np.array(res.evaluation.energy))
objective = Objective(np.array(res.evaluation.objective))
if res.evaluation.constraint_violations is not None:
    const_name, const_values  = list(res.evaluation.constraint_violations.items())[0]
    constraint_violation = ConstraintViolation(np.array(const_values), const_name)
else:
    constraint_violation = ConstraintViolation()

print(energy)
print(objective)
print(constraint_violation)

Energy(data=array([0.]), name='energy', children=None)
Objective(data=array([1.]), name='objective', children=None)
ConstraintViolation(data=array([0.]), name='onehot', children=None)


In [23]:
record = Record(
    name="0",
    children=[b_id, energy, objective, constraint_violation]
)


In [24]:
record.to_dict()

{'benchmark_id': '6e313955-bb9e-4b9d-b66b-860a2c14f291',
 'energy': array([0.]),
 'objective': array([1.]),
 'onehot': array([0.])}

In [25]:
record.to_series()

benchmark_id    6e313955-bb9e-4b9d-b66b-860a2c14f291
energy                                         [0.0]
objective                                      [1.0]
onehot                                         [0.0]
dtype: object

In [45]:
table = Table(children=[record])
table.to_dataframe()

,benchmark_id,energy,objective,onehot
0,6e313955-bb9e-4b9d-b66b-860a2c14f291,[0.0],[1.0],[0.0]


In [39]:
s1 = pd.Series([1, 2, 3], index=["a", "b", "c"])
s2 = pd.Series([1, 2, 3], index=["a", "b", "c"])

In [43]:
pd.concat([s1], axis=1).T

,a,b,c
0,1,2,3


a    1
b    2
c    3
dtype: int64